In [13]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GEMINI_API_KEY"] = ""

In [19]:
import os
import requests
import json
from typing import Any, List, Optional
from langchain_core.prompts import PromptTemplate
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult
from pydantic import BaseModel, Field, validator
from langchain_core.output_parsers import StrOutputParser


In [20]:

# --- 1. Custom LLM Implementation ---
class CustomHTTPGemini(BaseLLM):
    """
    A custom LangChain LLM wrapper that interacts with the Google Gemini API
    using direct HTTP requests (POST to generateContent endpoint).
    """

    # Model and API Configuration
    api_key: Optional[str] = None
    model_name: str = Field(default="gemini-2.5-flash", alias="model")
    # Base URL remains for configuration, though we construct the full endpoint in _call
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/models/"

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        # Ensure the API key is set, prioritizing the passed argument or environment variable
        if not self.api_key:
            self.api_key = os.getenv("GEMINI_API_KEY")

        if not self.api_key:
            raise ValueError("GEMINI_API_KEY must be provided or set as an environment variable.")

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "custom_http_gemini"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> str:
        """
        The core logic to make the HTTP POST request to the Gemini API.
        This method checks **kwargs for a pre-built 'request_data' payload.
        """

        print(f"\n--- LLM Invoked for prompt (truncated): {prompt[:50]}... ---")

        # 1. Construct the API Endpoint for the specific model and method
        api_endpoint = f"{self.base_url}{self.model_name}:generateContent"

        # 2. Construct the complete URL with API Key as query parameter
        url = f"{api_endpoint}?key={self.api_key}"

        # 3. Define the HTTP headers
        headers = {
            "Content-Type": "application/json"
        }

        # --- NEW LOGIC: Determine Request Payload ---
        request_data = kwargs.get("request_data")
        print(request_data)

        # if request_data:
        #     # Case A: Payload provided in kwargs (Constructed 'outside' _call)
        #     # We must inject the dynamic prompt into the provided structure.
        #     # We assume the user has provided a valid template with 'contents'.
        #     if 'contents' in request_data and request_data['contents'] and 'parts' in request_data['contents'][0]:
        #         print("Using custom request payload from **kwargs, injecting prompt.")
        #         # Insert the dynamic prompt into the designated text field
        #         request_data['contents'][0]['parts'][0]['text'] = prompt
        #     else:
        #         raise ValueError("Custom request_data must contain the structure: ['contents'][0]['parts'][0]['text']")
        # else:
        #     # Case B: Default payload construction (as in previous versions)
        #     print("Using default request payload construction.")
        #     request_data = {
        #         "contents": [
        #             {
        #                 "parts": [
        #                     {
        #                         "text": prompt
        #                     }
        #                 ]
        #             }
        #         ]
        #     }
        # --- END NEW LOGIC ---

        # 4. Send the request
        try:
            # Using 'json=request_data' is a cleaner way to send JSON data with requests
            response = requests.post(
                url=url,
                headers=headers,
                json=request_data
            )
            response.raise_for_status() # Raise exception for bad status codes

            response_json = response.json()

            # 5. Extract the generated text from the structured JSON response
            generated_text = response_json['candidates'][0]['content']['parts'][0]['text']

            return generated_text

        except requests.exceptions.HTTPError as err:
            error_message = f"Gemini API HTTP Error ({err.response.status_code}): {err.response.text}"
            raise RuntimeError(error_message) from err
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred during API call: {e}")

    # Note: _generate is required by BaseLLM
    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the LLM on a list of prompts."""
        generations = []
        for prompt in prompts:
            # Pass **kwargs through to _call
            text = self._call(prompt, stop, run_manager, **kwargs)
            generations.append([{"text": text}]) # Wrap the result in the expected structure
        return LLMResult(generations=generations)



In [21]:


# --- 2. LCEL Chain Integration and Demonstration ---
if __name__ == "__main__":
    # --- Setup ---
    print("--- LangChain Custom HTTP Gemini Example ---")

    # NOTE: Set your API Key in your environment before running:
    # export GEMINI_API_KEY="YOUR_API_KEY_HERE"

    # Initialize the custom LLM
    try:
        custom_llm = CustomHTTPGemini(model_name="gemini-2.5-flash")
    except ValueError as e:
        print(f"\nERROR: {e}")
        print("Please set the GEMINI_API_KEY environment variable and try again.")
        exit()

    # Define a prompt template
    prompt_template = PromptTemplate.from_template(
        "You are a helpful assistant. Answer the following question concisely: {question}"
    )

    # -----------------------------------------------------------
    # DEMONSTRATION 1: Standard LCEL Chain (Uses Default Payload Construction)
    # -----------------------------------------------------------
    # print("\n[DEMO 1] Running Standard LCEL Chain (Uses default payload inside _call)")

    # chain = prompt_template | custom_llm | StrOutputParser()
    # question_1 = "What is the capital of France?"

    # try:
    #     response_1 = chain.invoke({"question": question_1})
    #     print(f"\nQuestion 1: {question_1}")
    #     print(f"Model Response:\n{response_1}")
    # except Exception as e:
    #     print(f"Demo 1 failed: {e}")


    # -----------------------------------------------------------
    # DEMONSTRATION 2: Direct Call with Pre-constructed Payload
    # This simulates construction *outside* the _call method.
    # -----------------------------------------------------------
    print("\n\n[DEMO 2] Running Direct Call (Passes payload via kwargs)")

    question_2 = "What are the first three lines of 'Jabberwocky'?"

    # 1. Construct the payload structure outside the call (The argument you wanted passed)
    # IMPORTANT: The prompt text is empty here; it will be injected by the _call method.
    custom_request_payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": question_2 # The LLM will overwrite this with the actual prompt
                    }
                ]
            }
        ]
    }

    # 2. Define the final prompt string (What LCEL would normally produce)
    final_prompt_string = prompt_template.format(question=question_2)

    try:
        # 3. Call _call directly, passing the custom payload in **kwargs
        # NOTE: This bypasses the complexity of LangChain's .invoke,
        # but demonstrates the logic within _call().
        response_2 = custom_llm._call(
            prompt=final_prompt_string,
            request_data=custom_request_payload # Passed in **kwargs
        )
        print(f"\nQuestion 2: {question_2}")
        print(f"Model Response (Temp=0.3):\n{response_2}")
    except Exception as e:
        print(f"Demo 2 failed: {e}")


    print("\n--- End of Chain Execution ---")

--- LangChain Custom HTTP Gemini Example ---


[DEMO 2] Running Direct Call (Passes payload via kwargs)

--- LLM Invoked for prompt (truncated): You are a helpful assistant. Answer the following ... ---
{'contents': [{'parts': [{'text': "What are the first three lines of 'Jabberwocky'?"}]}]}

Question 2: What are the first three lines of 'Jabberwocky'?
Model Response (Temp=0.3):
The first three lines of 'Jabberwocky' are:

'Twas brillig, and the slithy toves
Did gyre and gimble in the wabe:
All mimsy were the borogoves,

--- End of Chain Execution ---
